In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('..')

In [5]:
# !wget http://ipsar.fesb.unist.hr/ipsar_data/heridal.zip
# !unzip ./heridal.zip

In [6]:
!ls ../heridal

patches  testImages  trainImages


In [7]:
import drone_sar
from drone_sar.ipsar_dataset import IPSARDataset
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from collections import Counter

In [8]:
train_data = IPSARDataset("../heridal/trainImages/")
test_data = IPSARDataset("../heridal/testImages")
item = test_data[0]

In [9]:
len(train_data)

1583

In [ ]:
# Counter([len(el["boxes"]) for el in train_data])

In [101]:
item.keys()

dict_keys(['img_path', 'pil', 'target_sizes', 'boxes', 'class_labels'])

In [11]:
item["boxes"].shape, item["class_labels"].shape

(torch.Size([2, 4]), torch.Size([2]))

In [12]:
# IPSARDataset.plt_show(item, size=10)

In [13]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

In [14]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    num_labels=1,
    ignore_mismatched_sizes=True,
)

/home/iz/.cache/pypoetry/virtualenvs/drone-sar-xLdq85T6-py3.10/lib/python3.10/site-packages/transformers/models/detr/image_processing_detr.py:776: FutureWarning: The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
  warnings.warn(
Some weights of DetrForObjectDetection were not initialized from the model checkpoint at facebook/detr-resnet-50 and are newly initialized because the shapes did not match:
- class_labels_classifier.weight: found shape torch.Size([92, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
- class_labels_classifier.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
train_dl = train_data.get_dataloader(processor, bs=10, shuffle=True)
test_dl = test_data.get_dataloader(processor, bs=10, shuffle=False)

In [28]:
batch, info = next(iter(train_dl))

In [30]:
# outputs = model(**batch)

In [35]:
# outputs["loss_dict"], outputs["loss"]

In [36]:
# results = processor.post_process_object_detection(outputs, target_sizes=info["target_sizes"], threshold=0.5)

In [37]:
import lightning as L
from lightning.pytorch import loggers as pl_loggers


class LightningDetector(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def training_step(self, input, batch_id):
        batch, info = input
        num_boxes = sum([len(l["boxes"]) for l in batch["labels"]])
        if num_boxes == 0:
            return None # Skip the optim step if no boxes present (we get an error otherwise)

        output = self.model(**batch)
        self.log("loss", output["loss"])
        self.log_dict(output["loss_dict"])

        return output["loss"]

In [38]:
lightnin_detector = LightningDetector(model)

In [39]:
tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs/")
trainer = L.Trainer(logger=tb_logger, max_epochs=10)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(lightnin_detector, train_dataloaders=train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                   | Params
-------------------------------------------------
0 | model | DetrForObjectDetection | 41.5 M
-------------------------------------------------
41.3 M    Trainable params
222 K     Non-trainable params
41.5 M    Total params
166.037   Total estimated model params size (MB)


Training: |                                                        | 0/? [00:00<?, ?it/s]